In [1]:
#Paquetes necesarios
import cv2
import numpy as np
from deepface import DeepFace
from PIL import Image

In [2]:
def overlay_image_alpha(img, overlay, position, alpha_mask):
    """Superpone una imagen `overlay` en la posición especificada sobre la `img`."""
    x, y = position
    h, w = overlay.shape[:2]

    # Recorte de las coordenadas para asegurar que no se salga del frame
    if x < 0:
        overlay = overlay[:, -x:]
        alpha_mask = alpha_mask[:, -x:]
        w += x
        x = 0
    if y < 0:
        overlay = overlay[-y:, :]
        alpha_mask = alpha_mask[-y:, :]
        h += y
        y = 0
    if x + w > img.shape[1]:
        overlay = overlay[:, :img.shape[1] - x]
        alpha_mask = alpha_mask[:, :img.shape[1] - x]
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        overlay = overlay[:img.shape[0] - y, :]
        alpha_mask = alpha_mask[:img.shape[0] - y, :]
        h = img.shape[0] - y

    # Crear un fondo y sobreponer con la máscara alfa
    img_crop = img[y:y+h, x:x+w]
    img_crop[:] = img_crop * (1 - alpha_mask[:, :, np.newaxis] / 255.0) + overlay * (alpha_mask[:, :, np.newaxis] / 255.0)

    return img

In [12]:
def load_gif_frames(gif_path):
    pil_gif = Image.open(gif_path)
    frames = []
    try:
        while True:
            frame = pil_gif.convert("RGBA")
            frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGBA2BGRA)
            frames.append(frame)
            pil_gif.seek(pil_gif.tell() + 1)
    except EOFError:
        pass
    return frames

dnn_model = "deploy.prototxt.txt"
dnn_weights = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(dnn_model, dnn_weights)
video_capture = cv2.VideoCapture(0)
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Cargar gráficos del cabello y aura GIF
hair = cv2.imread('pelo_super_saiyan.png', cv2.IMREAD_UNCHANGED)
aura_frames = load_gif_frames('aura.gif')  # Cargar fotogramas del GIF de aura
scouter = cv2.imread('scouter.png', cv2.IMREAD_UNCHANGED)
aura_frame_count = len(aura_frames)  # Número de fotogramas en el GIF
aura_index = 0  # Índice de fotograma inicial

# Procesar el video en tiempo real
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Detección de rostros
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104.0, 177.0, 123.0], False, False)
    net.setInput(blob)
    detections = net.forward()

    # Para cada cara detectada
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")
            face_width = x2 - x
            face_height = y2 - y

            # Detectar emoción con DeepFace
            try:
                face_img = frame[y:y2, x:x2]  # Extrae el rostro detectado
                analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                emotion = analysis[0]['dominant_emotion']

                # Activar efectos si la emoción es enfado
                if emotion == 'angry':
                    # Redimensionar gráficos de cabello
                    hair_resized = cv2.resize(hair, (face_width * 3, face_height * 2))

                    # Extraer canal alfa como máscara
                    hair_alpha = hair_resized[:, :, 3]

                    aura_frame = aura_frames[aura_index]
                    aura_resized = cv2.resize(aura_frame, (face_width * 10, frame_height))

                    # Actualizar el índice del GIF para la animación
                    aura_index = (aura_index + 1) % aura_frame_count

                    # Extraer canal alfa del fotograma del GIF
                    aura_alpha = aura_resized[:, :, 3]

                    scouter_resized = cv2.resize(scouter, (int(face_width * 0.6), int(face_height * 0.3)))
                    scouter_alpha = scouter_resized[:, :, 3]

                    hair_x = x + face_width // 2 - hair_resized.shape[1] // 2
                    hair_y = y - round(hair_resized.shape[0] * 0.7)
                    aura_x = x + face_width // 2 - aura_resized.shape[1] // 2
                    aura_y = y - face_height
                    eye_x = x - int(face_width * 0.10)
                    eye_y = y + int(face_height * 0.3)

                    # Superponer gráficos sobre el frame en posiciones específicas
                    frame = overlay_image_alpha(frame, hair_resized[:, :, :3], (hair_x, hair_y), hair_alpha)
                    frame = overlay_image_alpha(frame, scouter_resized[:, :, :3], (eye_x, eye_y), scouter_alpha)
                    frame = overlay_image_alpha(frame, aura_resized[:, :, :3], (aura_x, aura_y), aura_alpha)
                    

            except Exception as e:
                print("Error en análisis de emociones:", e)

    cv2.imshow('Filtro Saiyan', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

# Liberar la cámara y cerrar ventanas
video_capture.release()
cv2.destroyAllWindows()